# Installation
From [the docs](http://qcodes.github.io/Qcodes/start/index.html):

1. Install [Anaconda](https://www.anaconda.com/download/). Get the Python 3 version. No need to install the VSCode stuff.
1. Save [environment.yml](https://raw.githubusercontent.com/QCoDeS/Qcodes/master/environment.yml) to the machine.
1. Run the following in an Anaconda Prompt:
```
conda env create -f environment.yml
activate qcodes
pip install qcodes
```
1. Now you can run `jupyter notebook` from within your qcodes environment.

In [ ]:
import time
import numpy as np
import qcodes as qc
from qcodes.dataset.measurements import Measurement

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument). For demonstration purposes, here I just use dummy instruments `dac` and `dmm`.

In [ ]:
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import random
dmm.idc.get = lambda: random.gauss(1, 0.1)
dmm.ig.get = lambda: random.gauss(0, 0.01)

You can set `step` and `inter_delay` on any parameters. These will apply to all future attempts to set the value, including those in loops. For instance, if `dac.ch1` can be ramped at a maximum speed of 0.1 V / ms, then do this:

In [ ]:
dac.ch1.step = 0.1
dac.ch1.inter_delay = 0.001

# Experiment setup

The first time you run qcodes on a new computer, you need to create the SQLite database. See [the docs](https://qcodes.github.io/Qcodes/dataset/index.html) for details on the design of the dataset and database. You will then want to make a new experiment to hold your data. At the very least, make a new experiment when you change out your samples.

In [ ]:
qc.initialise_database()
qc.new_experiment(name='demo', sample_name='my best sample')

# Measurement

## 1D sweep

In [ ]:
%load sweep.py

In [ ]:
s = Sweep()
s.follow_param(dmm.ig, gain=10)
s.follow_param(dmm.idc)
s.sweep(dac.ch1, np.linspace(0, 25, 10), inter_delay=0.5)
s.sweep(dac.ch1, np.linspace(25, 0, 10), inter_delay=0.5)
s.sweep(dac.ch1, np.concatenate((np.linspace(0, 25, 10), np.linspace(25, 0, 10))), inter_delay=0.1)

## 2D sweep

In [ ]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.ig, setpoints=(dac.ch1, dac.ch2))

with meas.run() as datasaver:
    try:
        for set_ch1 in np.linspace(0, 25, 10):
            for set_ch2 in np.linspace(0, 10, 10):
                dac.ch1.set(set_ch1)
                dac.ch2.set(set_ch2)
                ig = dmm.ig.get()

                datasaver.add_result((dac.ch1, set_ch1), (dac.ch2, set_ch2), (dmm.ig, ig))
    except KeyboardInterrupt:
        pass

## Where is the data?

See the `get_data_by_id` docstring.

TODO: Make this easier to save to a csv or something.

In [ ]:
get_data_by_id(datasaver.run_id)